In [39]:
import spacy 
import import_ipynb
from spacy import displacy
import random
import categories
from categories import *
from categories import features_list
from categories import question_list
import polyfuzz as pf
import re

In [ ]:
#devo trovare tutte le dipendenze per una certa caratteristica dei padawain
class NLU: 
    def __init__(self, text):
        self.text = text
        self.nlp = spacy.load("en_core_web_sm")
        self.doc = self.nlp(self.text)
        #self.category = self.categories.categories
        #self.feature_list = self.category.get_features()
        #self.feature_list = set(self.categories.get_features())
        
        
    #cerca le dipendenze di tutte le parole (pos tagging)
    def find_dependency(self):
        dict_of_dependency = {}
        for token in self.doc:
            dict_of_dependency[token.text] = token.dep_, token.pos_, token.head.text, token.morph
        
        return dict_of_dependency

    #visualizza le dipendenze
    def visualize_dependency(self):
        displacy.render(self.doc, style="dep")
        return self.doc

    #vado a fare le regex per le domande. La regex deve andare a individuare 3 cose: eventuale punto di domande, la categoria e la feature, e la negazione eventualmente
    def regular_expression_questions(self):
        questions = [question.split() for question in question_list] #prendo la lista delle domande per ogni categoria
        features = [feature for feature in features_list] #prendo la lista delle features della specifica categoria
        found = [] #lista delle domande trovate
        pattern_1 = "[?]"  #primo pattern in cui vado a vedere se rappresenta una domanda
        #pattern_2 = f"{self.category}" #lista delle categorie
        pattern_3 = f"{features}"#lista delle caratteristiche

        for q in questions:
            x = re.search(pattern_1, q[0]) #vado a vedere se ce' il punto di domanda
            if x:
                #re.search(pattern_2, q[0]) #vado a vedere se c'e' la categoria
                re.search(pattern_3, q[0]) #vado a vedere se c'e' la feature
                found.append(q[0])

            else:
                found.append(None) #se non c'e' nessuna domanda allora metto None


    #Regex per le risposte. La regex deve andare a individuare la features presente nella risposta
    def regular_expression_answers(self):    
        features = [feature.split() for feature in features_list] #prendo la lista delle features della specifica categoria
        self.pos = True
        correct = [] #lista delle risposte corrette
        #pattern_1 = f"{self.category}" #lista delle categorie
        pattern_2 = f"{features}"#lista delle caratteristiche

        for token in self.doc:
            x = re.search(pattern_2, token.text) #vado a vedere la feature nel testo
            if x:
                correct.append(token.text)
            else:
                correct.append(None)


    
            
    #vado a controllare le dependency per le features dei padawain
    def check_dependency_padawian(self):
        check_list = [feature.split() for feature in features_list] #prendo la lista delle feature della specifica categoria
        correct = []

        for token in self.doc: #vado nel testo
            for f in check_list: #vado nella lista delle features
                if token.text == f[0]: #se la feature è uguale al token    
                    if len(f) == 1:
                        correct.append(token.text)
                    
                    #se e' una parola composta allora non va bene
                    elif token.i == len(self.doc) - 1 and len(f) > 1 :
                        correct.append(None)
                    
                    else:
                        for i in range(1, len(f)):
                            if token.nbor(i).text != f[i]:
                                correct.append(None)
                            
                            if token.i == len(self.doc) - 1 and len(f) > 1 :
                                correct.append(None)
                        
                        return "".join(f)
                            
        correct = [x for x in correct if x is not None]
        return correct
    
     #fare funzione che prende la risposta che ci si aspetta e la confronta con la risposta data dall'utente
    def check_response(self, response, correct_response):
        max = 0
        for answer in correct_response:
            #controllo della similiarità con polyfuzz
            model = pf.Model("TF-IDF")
            model.match([answer, response])
            print(model.get_matches())
            sim = model.get_matches().at[0,"Similarity"]
            if sim > max:
                max = sim

        return sim
    
    
    """
    def features_in_text(self):
       return len(self.check_dependency_padawian())
    """

#Praticamente cerco di capire se la frase che e' stata detta e' positiva o negativa. In questo modo nel dialog system gestisco la tipologia di risposta
"""
    def check_response_type(self):
        self.pos = True
        for token in self.doc:
            if token.text == 'no' or token.text == 'not' or "Neg" in token.morph.get("Polarity"):
                self.pos = False
            
        return self.pos
"""       
if __name__ == "__main__":
    sent = NLU("I am Humanoid and i have force and want to learn to use the lightsaber") #Piccola frase di esempio. In questo caso va a pescare dalla losta force e lightsaber
    print(sent.text)
    print("\n")
    #print("Visualize dependency: ", sent.visualize_dependency())
    dependencies = sent.find_dependency()

    for key, value in dependencies.items():
        print(f"{key}, {value}")
    
    print("\n")
    print(f"Parole chiave: {sent.check_dependency_padawian()}")
    print("\n")
    print(f"Numero Parole Chiave: {sent.features_in_text()}")
    #print(f"{sent.check_response_type()}")